In [ ]:
# Environment configuration
!nvidia-smi
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/PyTrial_dev_group/tutorials
%cd ./trial_simulation/sequence
%ls

Wed Nov 16 22:30:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Configure python environment

In [ ]:
# move working dir
import os
os.chdir('../..')

# configure the environment for PyTrial
!python --version
!pip install --upgrade pip
!pip install -r pytrial/requirements.txt

Python 3.7.15
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/RyanWangZf/transtab.git (to revision main) to /tmp/pip-req-build-vn2fpzof
  Running command git clone --filter=blob:none --quiet https://github.com/RyanWangZf/transtab.git /tmp/pip-req-build-vn2fpzof
  Resolved https://github.com/RyanWangZf/transtab.git to commit 267bb47c007e45b4854ef3678a5ca73097a1e71b
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/RyanWangZf/promptehr.git (to revision main) to /tmp/pip-req-build-clejn45s
  Running command git clone --filter=blob:none --quiet https://github.com/RyanWangZf/promptehr.git /tmp/pip-r

### **Step 1 & 2: Import the package and prepare input data**

In [ ]:
from pytrial.data.demo_data import load_synthetic_ehr_sequence
data = load_synthetic_ehr_sequence()

In [ ]:
from pytrial.tasks.trial_simulation.data import SequencePatient

train_data = SequencePatient(
    data={
        'v': data['visit'],
        'y': data['y'],
        'x': data['feature'],
        },
    metadata={
        'visit': {'mode': 'dense'},
        'label': {'mode': 'tensor'},
        'voc': data['voc'],
        'max_visit': 20,
        'n_num_feature': data['n_num_feature'],
        'cat_cardinalities': data['cat_cardinalities'],
    }
)

### **Step 3: Init the model using pretrained weights**

- `PromptEHR` is associated with pretrained weights on MIMIC-III data and can be used to generate synthetic directly.

In [ ]:
from pytrial.tasks.trial_simulation.sequence import PromptEHR

vocs = data['voc']
model = PromptEHR()
model.from_pretrained()

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'DataTokenizer'.


Load pretrained PromptEHR model from ./simulation/pretrained_promptEHR
Load the pre-trained model from: ./simulation/pretrained_promptEHR


### **Step 4: Synthetic data generation**

In [ ]:
synthetic_seqdata = model.predict(train_data, n=500, n_per_sample=1, verbose=True)

  2%|▏         | 11/500 [00:11<06:37,  1.23it/s]

522 reach model max length 512, do cut.
514 reach model max length 512, do cut.
524 reach model max length 512, do cut.
519 reach model max length 512, do cut.
517 reach model max length 512, do cut.
513 reach model max length 512, do cut.
527 reach model max length 512, do cut.
514 reach model max length 512, do cut.
517 reach model max length 512, do cut.
518 reach model max length 512, do cut.
522 reach model max length 512, do cut.
516 reach model max length 512, do cut.
515 reach model max length 512, do cut.
526 reach model max length 512, do cut.
518 reach model max length 512, do cut.


 75%|███████▍  | 373/500 [06:19<02:20,  1.11s/it]

513 reach model max length 512, do cut.


 75%|███████▍  | 374/500 [06:25<05:10,  2.46s/it]

520 reach model max length 512, do cut.


 93%|█████████▎| 465/500 [07:57<00:40,  1.16s/it]

517 reach model max length 512, do cut.


100%|██████████| 500/500 [08:42<00:00,  1.04s/it]


- Let's take a look at the generated synthetic data, it should have the same format as the input `train_data`.

In [ ]:
from torch.utils.data import DataLoader
from pytrial.data.patient_data import SeqPatientCollator

collate_fn = SeqPatientCollator(config={'visit_mode':'dense'})
loader = DataLoader(synthetic_seqdata, batch_size=2, collate_fn=collate_fn, num_workers=0)
loader = iter(loader)
batch = next(loader)
print(batch)

defaultdict(<class 'list'>, {'v': defaultdict(<class 'list'>, {'diag': [[[1, 7, 41, 269, 46, 49, 21, 664, 313, 7], [0, 49, 10, 69]], [[2, 69, 485, 135, 364, 13, 12, 47, 16, 14, 18, 20, 405, 630, 23, 24, 315, 15], [320, 0, 36, 485, 135, 39, 40, 364, 269, 47, 49, 630, 377, 187, 25]]], 'prod': [[[0, 584, 583], [1, 4]], [[66, 38, 39, 7, 265, 12, 15, 181, 118, 85, 91], [0, 85]]], 'med': [[[0, 2, 6, 7, 73, 10, 75, 44, 11, 12, 51, 19, 22, 26, 91], [0, 1, 3, 6, 7, 41, 44, 16, 22, 26, 91, 28]], [[0, 1, 3, 36, 5, 40, 11, 44, 14, 16, 29, 20, 22, 26, 27, 28, 61], [1, 33, 3, 5, 6, 7, 41, 44, 14, 22, 26, 27, 29]]]}), 'x': tensor([[-1.0202,  0.0000,  0.0000],
        [ 1.3394,  1.0000,  0.0000]], dtype=torch.float64), 'y': tensor([False,  True])})


In [ ]:
batch.keys()

dict_keys(['v', 'x', 'y'])

### **Step 5: Evaluate the generated synthetic records**

- `RNNPrivacyDetection` trains an RNN model to discriminate synthetic and real data (binary classification), the output score is the ROC-AUC of the prediction. The higher the AUC, the more private (less realistic) the records are.

In [ ]:
from pytrial.tasks.trial_simulation.sequence.evaluation import RNNPrivacyDetection, RNNUtilityDetection

print(RNNPrivacyDetection.compute(train_data, synthetic_seqdata, device='cuda:0'))

{'lr': 0.0001, 'weight_decay': 0.0001}
***** Running training *****
  Num examples = 1200
  Num Epochs = 10
  Total optimization steps = 190


Iteration:  37%|███▋      | 7/19 [00:00<00:00, 69.05it/s]


######### Train Loss 10 #########
0 0.6897 




Training Epoch:  10%|█         | 1/10 [00:00<00:05,  1.75it/s]


######### Eval 10 #########
auc: 0.5696


...


Iteration:  63%|██████▎   | 12/19 [00:00<00:00, 21.78it/s]


######### Eval 180 #########
auc: 0.6633

######### Train Loss 190 #########
0 0.6041 




Training Epoch: 100%|██████████| 10/10 [00:08<00:00,  1.20it/s]


######### Eval 190 #########
auc: 0.6787
Load best ckpt from `./checkpoints/best`.
Training completes.


0.6787337326998555


- Utility detection measures the predictive model trained on the real data and on the synthetic data. Will return the AUC of the both models.

In [ ]:
print(RNNUtilityDetection.compute(train_data, synthetic_seqdata, device='cuda:0'))

{'lr': 0.0005, 'weight_decay': 0}
***** Running training *****
  Num examples = 700
  Num Epochs = 50
  Total optimization steps = 300


Iteration:  50%|█████     | 3/6 [00:00<00:00, 23.20it/s]


######### Train Loss 10 #########
0 0.6639 




Training Epoch:   4%|▍         | 2/50 [00:01<00:25,  1.91it/s]


######### Eval 10 #########
auc: 0.3306


...

Iteration: 100%|██████████| 3/3 [00:00<00:00, 38.28it/s]

Training Epoch:  94%|█████████▍| 47/50 [00:06<00:00,  6.42it/s]


######### Train Loss 140 #########
0 0.0859 


######### Eval 140 #########
auc: 0.7055



Iteration: 100%|██████████| 3/3 [00:00<00:00, 39.00it/s]

Training Epoch: 100%|██████████| 50/50 [00:07<00:00,  7.10it/s]


######### Train Loss 150 #########
0 0.0482 


######### Eval 150 #########
auc: 0.7046
Load best ckpt from `./checkpoints/best`.
Training completes.


{'real-data-model-auc': 0.6670813817330211, 'syn-data-model-auc': 0.6949648711943793}


### **Step 6: Train the model**

- We can also train PromptEHR from scratch.

In [ ]:
# reset the training hyperparameters
model.update_config({'epoch':1, 'batch_size':2})

### Model Config ###
{'code_type': ['diag', 'prod', 'med'], 'n_num_feature': 1, 'cat_cardinalities': [2, 37], 'epoch': 1, 'batch_size': 2, 'eval_batch_size': 16, 'eval_step': 500, 'learning_rate': 5e-05, 'weight_decay': 0.0001}
### Training Args ###
TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=8,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=10,
eval_delay=0,
eval_steps=1000,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
great

In [ ]:
import torch
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
model.fit(train_data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/84358834e73de6a82c22cec1d90eb45ef4f6eba5/config.json
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2

RuntimeError: ignored

### **Step 7: Save the model**

In [ ]:
model.save_model('./save_model/promptehr')
model.load_model('./save_model/promptehr')

Configuration saved in ./save_model/promptehr/config.json


Save the trained model to: ./save_model/promptehr


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/84358834e73de6a82c22cec1d90eb45ef4f6eba5/config.json
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2

RuntimeError: ignored